In [58]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
%matplotlib inline

### os 
import os 
import sys

### datetimes 
from datetime import datetime, timedelta

### scipy 
import numpy as np 
import pandas as pd
import xarray as xr

### plotting 
from matplotlib import pyplot as plt
import seaborn as sns

In [60]:
import pathlib

HOME = pathlib.Path.home()
CWD = pathlib.Path.cwd() 

In [61]:
import climpred 

In [62]:
import xskillscore

In [63]:
sys.path.append(str(HOME.joinpath('research/Smart_Ideas/code/ml4seas')))

In [64]:
import utils

### read the GCMs 

In [66]:
var_name = 'SST'

In [67]:
ecmwf_path = pathlib.Path(f"/media/nicolasf/END19101/data/GCMs/processed/CDS/ECMWF/{var_name}/")

In [68]:
lfiles = list(ecmwf_path.glob(f"CDS_ECMWF_{var_name}_????-??_monthly_anomalies_1981_2010_clim.nc"))

In [69]:
lfiles.sort()

In [70]:
len(lfiles)

468

In [71]:
lfiles[0]

PosixPath('/media/nicolasf/END19101/data/GCMs/processed/CDS/ECMWF/SST/CDS_ECMWF_SST_1981-01_monthly_anomalies_1981_2010_clim.nc')

In [72]:
lfiles[-1]

PosixPath('/media/nicolasf/END19101/data/GCMs/processed/CDS/ECMWF/SST/CDS_ECMWF_SST_2019-12_monthly_anomalies_1981_2010_clim.nc')

In [73]:
dset_ecmwf = xr.open_mfdataset(lfiles, concat_dim='init_time', combine='nested')

In [74]:
dset_ecmwf

,Array,Chunk
Bytes,18.30 GB,39.10 MB
Shape,"(468, 6, 25, 181, 360)","(1, 6, 25, 181, 360)"
Count,1872 Tasks,468 Chunks
Type,float32,numpy.ndarray


In [75]:
dset_ecmwf = dset_ecmwf.drop('month')

In [76]:
dset_ecmwf

,Array,Chunk
Bytes,18.30 GB,39.10 MB
Shape,"(468, 6, 25, 181, 360)","(1, 6, 25, 181, 360)"
Count,1872 Tasks,468 Chunks
Type,float32,numpy.ndarray


In [93]:
dset_ecmwf = dset_ecmwf.rename({'init_time':'init','step':'lead'})

ValueError: cannot rename 'init_time' because it is not a variable or dimension in this dataset

In [95]:
dset = dset_ecmwf.mean('member')

In [96]:
NW_pole = dset.sel(lat=slice(-50.,-35), lon=slice(170.,190.)).mean('lat').mean('lon')

In [97]:
SE_pole = dset.sel(lat=slice(-57.5,-45.), lon=slice(220.,240.)).mean('lat').mean('lon')

In [98]:
var_name = 'sst'

In [99]:
NW_pole.to_dataframe()[[var_name]]

/home/nicolasf/anaconda3/envs/pangeo/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


sst
init       lead          
1981-01-01 0    -0.016830
           1     0.075908
           2     0.042530
           3     0.086581
           4    -0.008781
...                   ...
2019-12-01 1     0.468567
           2     0.549050
           3     0.523722
           4     0.465935
           5     0.403117

[2808 rows x 1 columns]

In [100]:
NW_pole = NW_pole.to_dataframe()[[var_name]]

SE_pole = SE_pole.to_dataframe()[[var_name]]

/home/nicolasf/anaconda3/envs/pangeo/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/home/nicolasf/anaconda3/envs/pangeo/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [101]:
NW_pole = NW_pole.unstack()

SE_pole = SE_pole.unstack() 


In [103]:
from scipy.stats import zscore

In [104]:
NW_pole = NW_pole.apply(zscore)

SE_pole = SE_pole.apply(zscore) 

In [105]:
spsd = NW_pole - SE_pole

In [106]:
spsd

sst                                                  
lead               0         1         2         3         4         5
init                                                                  
1981-01-01 -1.650138 -0.849884 -0.692798 -0.346852 -0.514465 -0.607220
1981-02-01 -0.470411  0.397058  0.685210  0.618374  0.306490  0.089003
1981-03-01  0.577558  1.402947  1.513869  0.897049  0.789533  0.629598
1981-04-01  1.108955  1.250874  1.025652  0.703495  0.554767  0.530327
1981-05-01  0.901880  0.773870  0.520559  0.383366  0.278569  0.403810
...              ...       ...       ...       ...       ...       ...
2019-08-01  0.785320  0.382392  0.141407 -0.472868 -0.234278 -0.134126
2019-09-01 -0.126482 -0.857629 -0.935446 -0.711708  0.499539  0.783680
2019-10-01 -1.117251 -1.309704 -1.523153 -0.740970 -0.440047  0.311004
2019-11-01 -0.455370 -0.667219 -0.246041  0.355999  0.607141  0.754324
2019-12-01  1.429479  0.254424  0.353387  0.399053  0.210544  0.224759

[468 rows x 6 columns]

In [107]:
spsd.columns = pd.MultiIndex.from_product([['SPSD'], range(spsd.shape[1])])

In [110]:
opath = pathlib.Path('/media/nicolasf/END19101/data/climate_indices/GCMs/SPSD_GCM/')

In [111]:
if not opath.exists(): 
    opath.mkdir(parents=True)

In [112]:
spsd.to_csv('/media/nicolasf/END19101/data/climate_indices/GCMs/SPSD_GCM/SPSD_ECMWF.csv')